In [1]:
@file:DependsOn("ai.timefold.solver:timefold-solver-core:1.11.0")
@file:DependsOn("com.fasterxml.jackson.module:jackson-module-kotlin:2.15.2")


In [2]:
import com.fasterxml.jackson.annotation.JsonFormat

@JsonFormat(shape = JsonFormat.Shape.ARRAY)
data class Location(
    val latitude: Double,
    val longitude: Double
) {
    fun calcEuclideanDistanceTo(other: Location): Double {
        val xDifference = latitude - other.latitude
        val yDifference = longitude - other.longitude
        return Math.sqrt(xDifference * xDifference + yDifference * yDifference)
    }
}


In [3]:
import ai.timefold.solver.core.api.domain.entity.PlanningEntity
import ai.timefold.solver.core.api.domain.variable.PlanningVariable
import ai.timefold.solver.core.api.domain.variable.PlanningId
import com.fasterxml.jackson.annotation.JsonProperty

data class Agent(
    @PlanningId
    val name: String,
    val homeLocation: Location,
    val skills: List<String>
)


In [4]:
data class Meeting(
    val name: String,
    val location: Location,
    @JsonProperty("required_skill")
    val requiredSkill: String,
    @JsonProperty("time_window")
    val timeWindow: List<Int>, // Assuming time window is represented as [startTime, endTime]
    val duration: Int
)


In [5]:
import ai.timefold.solver.core.api.domain.entity.PlanningEntity
import ai.timefold.solver.core.api.domain.variable.PlanningVariable

@PlanningEntity
data class Assignment(
    val meeting: Meeting
) {
    @PlanningVariable(valueRangeProviderRefs = ["agentRange"], nullable = true)
    var agent: Agent? = null

    // No-arg constructor required for Timefold
    constructor() : this(Meeting("", Location(0.0, 0.0), "", listOf(0, 0), 0))
}


In [7]:
import ai.timefold.solver.core.api.domain.solution.PlanningSolution
import ai.timefold.solver.core.api.domain.solution.PlanningEntityCollectionProperty
import ai.timefold.solver.core.api.domain.solution.ProblemFactCollectionProperty
import ai.timefold.solver.core.api.domain.solution.PlanningScore
import ai.timefold.solver.core.api.domain.valuerange.ValueRangeProvider
import ai.timefold.solver.core.api.score.buildin.hardsoft.HardSoftScore

@PlanningSolution
data class Schedule(
    @ProblemFactCollectionProperty
    @ValueRangeProvider(id = "agentRange")
    val agents: List<Agent>,

    @ProblemFactCollectionProperty
    val meetings: List<Meeting>,

    @PlanningEntityCollectionProperty
    val assignments: List<Assignment>
) {
    @PlanningScore
    var score: HardSoftScore? = null

    // No-arg constructor required for Timefold
    constructor() : this(emptyList(), emptyList(), emptyList())
}


In [8]:
import ai.timefold.solver.core.api.score.stream.Constraint
import ai.timefold.solver.core.api.score.stream.ConstraintFactory
import ai.timefold.solver.core.api.score.stream.ConstraintProvider
import ai.timefold.solver.core.api.score.buildin.hardsoft.HardSoftScore

class AssignmentConstraintProvider : ConstraintProvider {
    override fun defineConstraints(constraintFactory: ConstraintFactory): Array<Constraint> {
        return arrayOf(
            requiredSkillConstraint(constraintFactory),
            agentConflictConstraint(constraintFactory),
            minimizeTravelDistanceConstraint(constraintFactory),
            timeWindowConstraint(constraintFactory)
        )
    }

    // Hard constraint: Agent must have the required skill
    private fun requiredSkillConstraint(constraintFactory: ConstraintFactory): Constraint {
        return constraintFactory.from(Assignment::class.java)
            .filter { assignment ->
                val agentSkills = assignment.agent?.skills ?: return@filter false
                !agentSkills.contains(assignment.meeting.requiredSkill)
            }
            .penalize(HardSoftScore.ONE_HARD)
            .asConstraint("Agent must have the required skill")
    }

    // Hard constraint: An agent can be assigned to only one meeting at a time
    private fun agentConflictConstraint(constraintFactory: ConstraintFactory): Constraint {
        return constraintFactory.fromUniquePair(Assignment::class.java)
            .filter { a1, a2 ->
                a1.agent != null && a1.agent == a2.agent &&
                        timeOverlap(a1.meeting, a2.meeting)
            }
            .penalize(HardSoftScore.ONE_HARD)
            .asConstraint("Agent cannot be assigned to overlapping meetings")
    }

    // Soft constraint: Minimize total travel distance
    private fun minimizeTravelDistanceConstraint(constraintFactory: ConstraintFactory): Constraint {
        return constraintFactory.from(Assignment::class.java)
            .filter { it.agent != null }
            .penalizeDouble(HardSoftScore.ONE_SOFT) { assignment ->
                val agentLocation = assignment.agent!!.homeLocation
                val meetingLocation = assignment.meeting.location
                agentLocation.calcEuclideanDistanceTo(meetingLocation)
            }
            .asConstraint("Minimize travel distance")
    }

    // Hard constraint: Meeting must be within agent's availability (assuming agents are available all the time for now)
    // Alternatively, enforce meeting time windows
    private fun timeWindowConstraint(constraintFactory: ConstraintFactory): Constraint {
        // For simplicity, we assume agents are available at all times.
        // Implement this constraint if agents have specific availability windows.
        return constraintFactory.from(Assignment::class.java)
            .filter { it.agent != null } // Optional: Add time window constraints here
            .reward(HardSoftScore.ZERO)
            .asConstraint("Time window constraint placeholder")
    }

    // Helper function to check time overlap between two meetings
    private fun timeOverlap(meeting1: Meeting, meeting2: Meeting): Boolean {
        val start1 = meeting1.timeWindow[0]
        val end1 = meeting1.timeWindow[1]
        val start2 = meeting2.timeWindow[0]
        val end2 = meeting2.timeWindow[1]
        return start1 < end2 && start2 < end1
    }
}


Line_8.jupyter.kts (18:34 - 38) 'from(Class<A!>!): UniConstraintStream<A!>!' is deprecated. Deprecated in Java
Line_8.jupyter.kts (28:34 - 38) 'from(Class<A!>!): UniConstraintStream<A!>!' is deprecated. Deprecated in Java
Line_8.jupyter.kts (31:22 - 43) Type mismatch: inferred type is ([Error type: Cannot infer a lambda parameter type], [Error type: Cannot infer a lambda parameter type]) -> ??? but BiConstraintCollector<Line_3_jupyter.Agent!, Line_5_jupyter.Assignment!, TypeVariable(ResultContainerA_)!, TypeVariable(ResultA_)!>! was expected
Line_8.jupyter.kts (31:24 - 29) Cannot infer a type for this parameter. Please specify it explicitly.
Line_8.jupyter.kts (31:31 - 32) Cannot infer a type for this parameter. Please specify it explicitly.
Line_8.jupyter.kts (31:45 - 93) Type mismatch: inferred type is ([Error type: Cannot infer a lambda parameter type], [Error type: Cannot infer a lambda parameter type]) -> ??? but BiConstraintCollector<Line_3_jupyter.Agent!, Line_5_jupyter.Assignme

In [ ]:
import java.io.File
import com.fasterxml.jackson.module.kotlin.jacksonObjectMapper
import com.fasterxml.jackson.module.kotlin.readValue

val mapper = jacksonObjectMapper()

// Read the JSON data
data class InputData(
    val vehicles: List<Agent>,
    val visits: List<Meeting>
)

val inputData: InputData = mapper.readValue(File("data.json"))

// Create assignments (initially unassigned)
val assignments = inputData.visits.map { Assignment(it) }

// Create the Schedule (planning solution)
val schedule = Schedule(
    agents = inputData.vehicles,
    meetings = inputData.visits,
    assignments = assignments
)


In [ ]:
import ai.timefold.solver.core.config.solver.SolverConfig
import ai.timefold.solver.core.api.solver.SolverFactory
import ai.timefold.solver.core.api.solver.Solver

val solverConfig = SolverConfig()
    .withSolutionClass(Schedule::class.java)
    .withEntityClasses(Assignment::class.java)
    .withConstraintProviderClass(AssignmentConstraintProvider::class.java)
    .withTerminationSpentLimit(java.time.Duration.ofSeconds(30)) // Adjust as needed

val solverFactory = SolverFactory.create(solverConfig)
val solver = solverFactory.buildSolver()

// Create initial problem
val schedule = Schedule(assignments, agents)

// Solve the problem
println("Solving the problem...")
val solution = solver.solve(schedule)
println("Solving finished with score ${solution.score}")


In [ ]:
solution.assignments.filter { it.agent != null }.forEach { assignment ->
    println("${assignment.meeting.name} is assigned to ${assignment.agent!!.name}")
}


In [ ]:
// Prepare data for output
data class AssignmentOutput(
    val meetingName: String,
    val agentName: String,
    val meetingLocation: Location,
    val agentHomeLocation: Location,
    val requiredSkill: String,
    val agentSkills: List<String>,
    val timeWindow: Pair<Int, Int>,
    val duration: Int
)

val assignmentOutputs = solution.assignments.filter { it.agent != null }.map { assignment ->
    AssignmentOutput(
        meetingName = assignment.meeting.name,
        agentName = assignment.agent!!.name,
        meetingLocation = assignment.meeting.location,
        agentHomeLocation = assignment.agent!!.homeLocation,
        requiredSkill = assignment.meeting.requiredSkill,
        agentSkills = assignment.agent!!.skills,
        timeWindow = assignment.meeting.timeWindow,
        duration = assignment.meeting.duration
    )
}

// Write to JSON
val outputFile = File("assignments.json")
mapper.writeValue(outputFile, assignmentOutputs)
